In [2]:
#Import required dependencies
import pandas as pd
import numpy as np
import requests
import json
import warnings
warnings.filterwarnings('ignore')



JSONDecodeError: ignored

#Pull JSON data

url = 'https://api.stocktwits.com/api/2/streams/symbol/EURUSD.json'

data = requests.get(url)
# Convert json into dictionary
response_dict = data.json()

# Pretty Printing JSON string back
print(json.dumps(response_dict, indent=4, sort_keys=True))

In [17]:
data

<Response [403]>

In [6]:
#Loop through and add elements to dataframe
df = pd.DataFrame()
for i in range(0,len(data['messages'])):
    json_entry = data['messages'][i]
    followers = json_entry['user']['followers']
    try:
        sentiment = json_entry['entities']['sentiment']['basic']
    except:
        sentiment = 'None'
    date_created = json_entry['created_at']
    row = np.asarray([date_created,followers,sentiment])
    row_df = pd.DataFrame(columns = ['Date Created','Followers','Sentiment'])
    df = df.append(pd.DataFrame(row).T)

#Misc - Tidy up df, rename cols, change data types
df = df.rename(columns = {0:'Date Created',1:'Followers',2:'Sentiment'})
df['Sentiment Value'] = df['Sentiment'].replace({'Bullish':1,'None':0,'Bearish':-1})
df['Followers'] = df['Followers'].astype(np.int32)
df['Date Created'] = pd.to_datetime(df['Date Created'])
df = df.reset_index().drop(['index'],axis = 1)

TypeError: ignored

In [ ]:
shortened_df = df[df['Sentiment Value'].isin([1,-1])]
#shortened_df = df
metric = (shortened_df['Followers'] * shortened_df['Sentiment Value']).sum()/shortened_df['Followers'].sum()
print('Sentiment Metric = {}'.format(round(metric,5)))

In [ ]:
#Calculating the alternate sentiment metric
lower = shortened_df['Followers'].max()*0.33
upper = shortened_df['Followers'].max()*0.66

for i,r in shortened_df.iterrows():
    if r['Followers'] < lower:
        shortened_df.loc[i,'Follower Group'] = 1
    elif r['Followers'] >= lower and r['Followers'] <= upper:
        shortened_df.loc[i,'Follower Group'] = 2
    else:
        shortened_df.loc[i,'Follower Group'] = 3

alt_metric = (shortened_df['Follower Group'] * shortened_df['Sentiment Value']).sum() / shortened_df['Follower Group'].sum()

print('Alternate Sentiment Metric = {}'.format (round(alt_metric,5)))